In [1]:
%pwd

'/Users/ryandevera/data-science/tesorio/NCVX-Neural-Structural-Optimization/notebooks'

In [2]:
%cd ..

/Users/ryandevera/data-science/tesorio/NCVX-Neural-Structural-Optimization


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import time
import torch
import os

# import problems to solve
import problems
import experiments
import train
import topo_api
import topo_physics
import models

from pygranso.private.getNvar import getNvarTorch
from pygranso.pygranso import pygranso
from pygranso.pygransoStruct import pygransoStruct

from scipy.ndimage import gaussian_filter

# torch.random.manual_seed(89)

# Running neural structural optimization with PyGranso 🧨

To check the validity of our new `torch` based code we will test the structural optimization with `torch.optim` optimizers.

### MBB Beam

The first problem that we will run is with the **MBB BEAM**.

In [4]:
def constrained_structural_optimization_function(model, ke, args, designs, losses):
    """
    Combined function for PyGranso for the structural optimization
    problem. The inputs will be the model that reparameterizes x as a function
    of a neural network. V0 is the initial volume, K is the global stiffness
    matrix and F is the forces that are applied in the problem.
    """
    # Initialize the model
    # In my version of the model it follows the similar behavior of the
    # tensorflow repository and only needs None to initialize and output
    # a first value of x
    logits = model(None)

    # kwargs for displacement
    kwargs = dict(
        penal=torch.tensor(args["penal"]),
        e_min=torch.tensor(args["young_min"]),
        e_0=torch.tensor(args["young"]),
    )
    x_phys = torch.sigmoid(logits)

    # Calculate the forces
    forces = topo_physics.calculate_forces(x_phys, args)

    # Calculate the u_matrix
    u_matrix, _ = topo_physics.sparse_displace(
        x_phys, ke, args, forces, args["freedofs"], args["fixdofs"], **kwargs
    )

    # Calculate the compliance output
    compliance_output, _, _ = topo_physics.compliance(x_phys, u_matrix, ke, args, **kwargs)

    # The loss is the sum of the compliance
    f = torch.abs(torch.sum(compliance_output))

    # Run this problem with no inequality constraints
    ci = None

    # Run this problem with no equality constraints
    ce = pygransoStruct()
    ce.c1 = 5e3 * torch.abs(torch.mean(x_phys) - args['volfrac'])

    # Append updated physical density designs
    designs.append(
        x_phys
    )  # noqa

    return f, ci, ce

In [ ]:
# Identify the problem
problem = problems.PROBLEMS_BY_NAME['multistory_building_32x64_0.5']

# Get the problem args
args = topo_api.specified_task(problem)
cnn_kwargs = None

# Trials
trials = []

for _ in range(25):

    # Initialize the CNN Model
    if cnn_kwargs is not None:
        cnn_model = models.CNNModel(args, **cnn_kwargs)
    else:
        cnn_model = models.CNNModel(args)

    # Put the cnn model in training mode
    cnn_model.train()

    # Create the stiffness matrix
    ke = topo_physics.get_stiffness_matrix(
        young=args["young"],
        poisson=args["poisson"],
    )

    # Create the combined function and structural optimization
    # setup
    # Save the physical density designs & the losses
    designs = []
    losses = []
    # Combined function
    comb_fn = lambda model: constrained_structural_optimization_function(  # noqa
        model, ke, args, designs, losses
    )

    # Initalize the pygranso options
    opts = pygransoStruct()

    # Set the device
    opts.torch_device = torch.device('cpu')

    # Setup the intitial inputs for the solver
    nvar = getNvarTorch(cnn_model.parameters())
    opts.x0 = (
        torch.nn.utils.parameters_to_vector(cnn_model.parameters())
        .detach()
        .reshape(nvar, 1)
    )

    # Additional pygranso options
    opts.limited_mem_size = 20
    opts.double_precision = True
    opts.mu0 = 1.0
    opts.maxit = 500
    opts.print_frequence = 10
    opts.stat_l2_model = False
    opts.viol_eq_tol = 1e-8
    opts.opt_tol = 1e-8

    # Train pygranso
    start = time.time()
    soln = pygranso(var_spec=cnn_model, combined_fn=comb_fn, user_opts=opts)
    end = time.time()
    
    # Final structure
    pygranso_structure = designs[-1].detach().numpy()
    final_objective = soln.final.f
    
    trials.append((final_objective, pygranso_structure))

/Users/ryandevera/data-science/tesorio/NCVX-Neural-Structural-Optimization/models.py:70: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self.h = args["nely"] // total_resize
/Users/ryandevera/data-science/tesorio/NCVX-Neural-Structural-Optimization/models.py:71: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  self.w = args["nelx"] // tot



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                                             ║ 
Version 1.2.0                                                                                                    ║ 
Licensed under the AGPLv3, Copyright (C) 2021-2022 Tim Mitchell and Buyun Liang                                  ║ 
═════════════════════════════════════════════

/Users/ryandevera/.pyenv/versions/3.9.11/lib/python3.9/site-packages/pygranso/private/bfgsHessianInverseLimitedMem.py:237: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:2985.)
  alpha[j,:]  = self.rho[0,j] * (self.S[:,j].T  @ q)


   2 ║ 1.000000 │  145.700936059 ║  115.488056630 ║   -  │ 30.21288 ║ S  │     1 │ 1.000000 ║     1 │ 0.006457   ║ 
   3 ║ 1.000000 │  110.108390399 ║  110.042108505 ║   -  │ 0.066282 ║ S  │     1 │ 1.000000 ║     1 │ 0.002945   ║ 
   4 ║ 1.000000 │  104.240513516 ║  103.642694761 ║   -  │ 0.597819 ║ S  │     2 │ 2.000000 ║     1 │ 0.004359   ║ 
   5 ║ 1.000000 │  100.849531544 ║  100.367353315 ║   -  │ 0.482178 ║ S  │     1 │ 1.000000 ║     1 │ 0.004354   ║ 
   6 ║ 1.000000 │  99.7378997861 ║  99.7201276369 ║   -  │ 0.017772 ║ S  │     1 │ 1.000000 ║     1 │ 0.005469   ║ 
   7 ║ 0.478297 │  47.3086334062 ║  95.6009671107 ║   -  │ 1.582987 ║ S  │     2 │ 0.500000 ║     1 │ 0.007225   ║ 
   8 ║ 0.478297 │  45.8586705349 ║  95.0028823260 ║   -  │ 0.419086 ║ S  │     1 │ 1.000000 ║     1 │ 0.010346   ║ 
   9 ║ 0.478297 │  45.3031082098 ║  94.5595596605 ║   -  │ 0.075564 ║ S  │     1 │ 1.000000 ║     1 │ 0.008822   ║ 
  10 ║ 0.478297 │  45.1967875995 ║  94.2448072367 ║   -  │ 0.119788 ║ S 

  61 ║ 0.027813 │  2.28393903588 ║  81.9685271329 ║   -  │ 0.004162 ║ S  │     3 │ 0.250000 ║     1 │ 0.005892   ║ 
  62 ║ 0.027813 │  2.28045876775 ║  81.9465682077 ║   -  │ 0.001292 ║ S  │     2 │ 0.500000 ║     1 │ 0.001976   ║ 
  63 ║ 0.022528 │  1.84739949249 ║  81.9431952423 ║   -  │ 0.001350 ║ S  │    11 │ 9.77e-04 ║     1 │ 0.013171   ║ 
  64 ║ 0.022528 │  1.84642645693 ║  81.8890440220 ║   -  │ 0.001597 ║ S  │     5 │ 0.062500 ║     1 │ 0.016793   ║ 
  65 ║ 0.022528 │  1.84637571308 ║  81.8646004424 ║   -  │ 0.002097 ║ S  │     5 │ 0.062500 ║     1 │ 0.007236   ║ 
  66 ║ 0.022528 │  1.84296065040 ║  81.5647794394 ║   -  │ 0.005437 ║ S  │     2 │ 2.000000 ║     1 │ 0.002698   ║ 
  67 ║ 0.018248 │  1.49006399134 ║  81.4786377456 ║   -  │ 0.003242 ║ S  │     9 │ 0.003906 ║     1 │ 0.068125   ║ 
  68 ║ 0.018248 │  1.48937346483 ║  81.4322146117 ║   -  │ 0.003398 ║ S  │     5 │ 0.062500 ║     1 │ 0.156017   ║ 
  69 ║ 0.018248 │  1.48793249010 ║  81.2991452891 ║   -  │ 0.004385 ║ S 

...



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
═════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: A PyTorch-enabled port of GRANSO with auto-differentiation                                             ║ 
Version 1.2.0                                                                                                    ║ 
Licensed under the AGPLv3, Copyright (C) 2021-2022 Tim Mitchell and Buyun Liang                                  ║ 
═════════════════════════════════════════════

  41 ║ 0.166772 │  9.84621287374 ║  58.7914241268 ║   -  │ 0.041460 ║ S  │     1 │ 1.000000 ║     1 │ 0.021474   ║ 
  42 ║ 0.166772 │  9.84408923649 ║  58.7536909915 ║   -  │ 0.045629 ║ S  │     7 │ 0.046875 ║     1 │ 0.015103   ║ 
  43 ║ 0.166772 │  9.82403469246 ║  58.6233922950 ║   -  │ 0.047305 ║ S  │     1 │ 1.000000 ║     1 │ 0.002067   ║ 
  44 ║ 0.166772 │  9.81925857638 ║  58.3106926634 ║   -  │ 0.094678 ║ S  │     1 │ 1.000000 ║     1 │ 0.006269   ║ 
  45 ║ 0.166772 │  9.79370799563 ║  57.4984468909 ║   -  │ 0.204588 ║ S  │     7 │ 0.109375 ║     1 │ 0.005751   ║ 
  46 ║ 0.166772 │  9.64122187205 ║  57.3899453612 ║   -  │ 0.070196 ║ S  │     1 │ 1.000000 ║     1 │ 0.001482   ║ 
  47 ║ 0.166772 │  9.57200843247 ║  57.1700938916 ║   -  │ 0.037648 ║ S  │     1 │ 1.000000 ║     1 │ 0.002348   ║ 
  48 ║ 0.166772 │  9.56303450292 ║  57.1626665884 ║   -  │ 0.029913 ║ S  │     6 │ 0.156250 ║     1 │ 0.012808   ║ 
  49 ║ 0.166772 │  9.55256847100 ║  57.0147114814 ║   -  │ 0.044121 ║ S 

 101 ║ 0.088629 │  4.79338339608 ║  54.0380092460 ║   -  │ 0.004028 ║ S  │     2 │ 0.500000 ║     1 │ 0.004032   ║ 
 102 ║ 0.088629 │  4.77815990888 ║  53.9022858049 ║   -  │ 8.34e-04 ║ S  │     4 │ 0.125000 ║     1 │ 0.118140   ║ 
 103 ║ 0.088629 │  4.76795653930 ║  53.7453102184 ║   -  │ 0.004543 ║ S  │     4 │ 1.750000 ║     1 │ 0.003747   ║ 
 104 ║ 0.088629 │  4.76794914315 ║  53.7450371578 ║   -  │ 0.004560 ║ S  │    13 │ 2.44e-04 ║     2 │ 0.001152   ║ 
 105 ║ 0.088629 │  4.76487885460 ║  53.6820227814 ║   -  │ 0.007074 ║ S  │     1 │ 1.000000 ║     1 │ 8.10e-04   ║ 
 106 ║ 0.088629 │  4.76319708131 ║  53.6066242198 ║   -  │ 0.012075 ║ S  │     4 │ 0.375000 ║     1 │ 0.006443   ║ 
 107 ║ 0.088629 │  4.75482467225 ║  53.6036128147 ║   -  │ 0.003970 ║ S  │     1 │ 1.000000 ║     1 │ 0.007545   ║ 
 108 ║ 0.088629 │  4.75253798187 ║  53.4856485338 ║   -  │ 0.012138 ║ S  │     3 │ 1.500000 ║     1 │ 0.015571   ║ 
 109 ║ 0.088629 │  4.74228003940 ║  53.2568717464 ║   -  │ 0.022156 ║ S 

 161 ║ 0.003043 │  0.15742882603 ║  51.7158939607 ║   -  │ 4.43e-05 ║ S  │    10 │ 0.251953 ║     1 │ 1.63e-04   ║ 
 162 ║ 0.003043 │  0.15742744255 ║  51.7150776914 ║   -  │ 4.54e-05 ║ S  │     7 │ 0.015625 ║     2 │ 2.53e-04   ║ 
 163 ║ 0.003043 │  0.15742618278 ║  51.7143923068 ║   -  │ 4.62e-05 ║ S  │     9 │ 0.027344 ║     3 │ 6.17e-05   ║ 
 164 ║ 0.003043 │  0.15742594196 ║  51.7143528001 ║   -  │ 4.61e-05 ║ S  │     7 │ 0.015625 ║     4 │ 5.93e-05   ║ 
 165 ║ 0.003043 │  0.15742418241 ║  51.7133528121 ║   -  │ 4.74e-05 ║ S  │     9 │ 0.011719 ║     5 │ 2.71e-04   ║ 
 166 ║ 0.003043 │  0.15740899199 ║  51.7068660593 ║   -  │ 5.19e-05 ║ S  │     2 │ 0.500000 ║     1 │ 3.09e-04   ║ 
 167 ║ 0.003043 │  0.15740668851 ║  51.7051839374 ║   -  │ 5.47e-05 ║ S  │     7 │ 0.015625 ║     2 │ 6.92e-05   ║ 
 168 ║ 0.003043 │  0.15737961475 ║  51.7021058568 ║   -  │ 3.70e-05 ║ S  │     3 │ 0.750000 ║     2 │ 9.17e-05   ║ 
 169 ║ 0.003043 │  0.15733024338 ║  51.6801147443 ║   -  │ 5.46e-05 ║ S 

 221 ║ 1.97e-04 │  0.01015396184 ║  51.6398972267 ║   -  │ 1.61e-07 ║ S  │     3 │ 0.250000 ║    17 │ 7.90e-05   ║ 
 222 ║ 1.97e-04 │  0.01015392098 ║  51.6394120472 ║   -  │ 2.16e-07 ║ S  │    10 │ 0.001953 ║    14 │ 2.07e-04   ║ 
 223 ║ 1.97e-04 │  0.01015388854 ║  51.6391873811 ║   -  │ 2.27e-07 ║ S  │     4 │ 0.125000 ║    14 │ 3.07e-05   ║ 
 224 ║ 1.97e-04 │  0.01015386985 ║  51.6389148418 ║   -  │ 2.62e-07 ║ S  │     6 │ 0.031250 ║    13 │ 8.64e-05   ║ 
 225 ║ 1.97e-04 │  0.01015376516 ║  51.6385852737 ║   -  │ 2.22e-07 ║ S  │    14 │ 0.014038 ║    14 │ 1.86e-05   ║ 
 226 ║ 1.97e-04 │  0.01015352694 ║  51.6379397602 ║   -  │ 1.11e-07 ║ S  │     4 │ 0.875000 ║    13 │ 1.43e-05   ║ 
 227 ║ 1.97e-04 │  0.01015349844 ║  51.6378519802 ║   -  │ 9.99e-08 ║ S  │    10 │ 0.025391 ║    13 │ 5.37e-05   ║ 
 228 ║ 1.97e-04 │  0.01015349788 ║  51.6378437223 ║   -  │ 1.01e-07 ║ S  │    13 │ 7.32e-04 ║    14 │ 6.99e-05   ║ 
 229 ║ 1.97e-04 │  0.01015336271 ║  51.6374170723 ║   -  │ 4.97e-08 ║ S 

In [ ]:
best_trial = sorted(trials)[1]
best_trial[0]

In [ ]:
from scipy.ndimage import gaussian_filter

# pygranso_structure = gaussian_filter(best_trial[1], sigma=1.5)
pygranso_structure = best_trial[1]

# Plot the two structures together
fig, ax1 = plt.subplots(1, 1, figsize=(8, 5))

# pygranso
ax1.imshow(pygranso_structure, cmap='Greys')
ax1.grid()
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_title('Multi-story Building 32x64 Density 0.5')
fig.tight_layout()